In [ ]:
# LIBRARIES IMPORT
import mercury as mr
import ee
import geemap.foliumap as geemap
import os
import datetime
import webbrowser
import logging
import folium
from folium import plugins
import matplotlib.pyplot as plt
from folium.plugins import Draw
from ipyleaflet import *
from colorama import Fore
from PIL import Image
import concurrent.futures
import shutil

# PROGRAM MODULES IMPORT
sys.path.insert(0, 'gee//modules//') # gee modules path
sys.path.insert(0, 'osm//') # osm modules path
sys.path.insert(0, 'common//') # common files 
import create_ee_creds as cec
import txtreading as txtr
import landsat_init as l_init
import sentinel_init as s_init
import modis_init as m_init
import othersds_init as o_init
import customds_init as cd_init
import osm_processing as op
import welcome_processing as wp

In [ ]:
# PROGRAM PRE-PROCESSING

today = datetime.datetime.today()
time = today.strftime("%Y%m%d-%H.%M.%S")
timestr = str(time)

logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
handler = logging.FileHandler('gee//gaia-gee-errors.log')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

mr.Markdown(text = '**🌎 GAIA - Geospatial & Aerial Images Analyser [BETA] 🌎**')
mr.Markdown(text = '**🌎 This program solution is not final and some details are potentially to be changed 🌎**')
mr.Markdown(text = '**🌏 Current version - 0.51b 🌏**')

text = 'OSINT-TECHNOLOGIES'
text_params = {
    'fontsize': 10,
    'fontcolor': 'black',
    'bold': True,
    'padding': '3px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '1px',
    'position': 'bottomleft',
}

sources_list = ["Welcome to GAIA", "Google EE", "OpenStreetMap"]
control_panel_choice = mr.Select(value="Welcome to GAIA", choices=sources_list, label="Choose the source control panel 🔍")

app  = mr.App(title="GAIA v0.51b [BETA] 🌐",
              description="Open-Source Geospatial & Aerial Images Analyser",
              show_code=False,
              show_prompt=False,
              continuous_update=True,
              static_notebook=False,
              show_sidebar=True,
              full_screen=True,
              allow_download=False)

In [ ]:
if control_panel_choice.value == "Welcome to GAIA":
    current_directory = os.getcwd()
    cache_path = current_directory + "\cache"
    logs_path = current_directory + "\gee\gaia-gee-errors.log"

    ee_reg = mr.Button(label="Create GEE account")
    ee_service_acc = mr.Button(label="Create & Manage GEE service account")
    clr_cache_and_logs = mr.Button(label="Clear GAIA cache folder")
    mr.Markdown('**PROGRAM DETAILS**')
    print(Fore.GREEN + "Current GAIA directory path: {}".format(current_directory))
    print(Fore.GREEN + f"The size of the logs file is {os.path.getsize(logs_path) / (1024 * 1024)} MB.")
    print(Fore.GREEN + f"The size of the cache folder is {wp.get_folder_size(cache_path) / (1024 * 1024)} MB.")

    if ee_reg.clicked:
        webbrowser.open_new_tab('https://code.earthengine.google.com/register')
    
    if ee_service_acc.clicked:
        webbrowser.open_new_tab('https://console.cloud.google.com/iam-admin/serviceaccounts/')

    if clr_cache_and_logs.clicked:
        try:
            shutil.rmtree(cache_path)
        except FileNotFoundError:
            print(Fore.RED + "Sorry, but cache folder was not found in GAIA directory. You can track this folder's volume in the line above")

    print('\n'*2)
    txtr.ee_guide_read()
    txtr.osm_guide_read()

In [ ]:
# GOOGLE EE PROCESSING

if control_panel_choice.value == "Google EE":
    if str((cec.service_acc_filename).rstrip()) not in os.listdir():
        print(Fore.RED + '[GEE file/directory missing error]. Your EE credentials .JSON file was not found in GAIA directory')
        mr.Stop()
    else:
        key_file = (cec.service_acc_filename).rstrip()
        service_account = (cec.service_acc_mail).rstrip()
        credentials = ee.ServiceAccountCredentials(service_account, key_file)
        ee.Initialize(credentials)

    ds_info = mr.Checkbox(value=False, label='Show datasets knowledge base')

    date_one = mr.Text(value="2022-01-01", label="Enter start date (format YYYY-MM-DD)", rows=1)
    date_two = mr.Text(value="2022-01-02", label="Enter final date (format YYYY-MM-DD)", rows=1)

    max_cloud_covering = mr.Slider(value=80, label='Max cloud cover (%, may affect datasets quality)', min=0, max=100)
    datasets_landsat = mr.Checkbox(value=False, label="Show Landsat datasets")
    datasets_sentinel = mr.Checkbox(value=False, label="Show Sentinel datasets")
    datasets_modis = mr.Checkbox(value=False, label="Show MODIS datasets")
    datasets_other = mr.Checkbox(value=False, label='Show other datasets')
    manual_input = mr.Checkbox(value=False, label='Open custom GEE dataset menu')

    mini_map = mr.Checkbox(value=False, label='Show mini map')

    m = geemap.Map()
    m.add_basemap('HYBRID')
    m.add_basemap('TERRAIN')
    if mini_map.value:
        plugins.MiniMap().add_to(m)
                         
    try:
        if datasets_landsat.value:
            landsat = l_init.LandsatInitialization(m, date_one, date_two, max_cloud_covering)
            with concurrent.futures.ThreadPoolExecutor() as landsat_caller:
                landsat_caller.submit(landsat.ls7sr_init)
                landsat_caller.submit(landsat.ls8toa_init)
                landsat_caller.submit(landsat.ls8rawtc_init)
                landsat_caller.submit(landsat.ls8sr_init)
                landsat_caller.submit(landsat.ls9_init)
                landsat_caller.submit(landsat.ls9rawtc_init)
                landsat_caller.submit(landsat.ls9toatc_init)
    
        if datasets_sentinel.value:
            sentinel = s_init.SentinelInitialization(m, date_one, date_two, max_cloud_covering)
            with concurrent.futures.ThreadPoolExecutor() as sentinel_caller:
                sentinel_caller.submit(sentinel.sen2msisr_init)
                sentinel_caller.submit(sentinel.sen2msitoa_init)
                sentinel_caller.submit(sentinel.sen5pc_init)                

        if datasets_modis.value:
            modis = m_init.ModisInitialization(m, date_one, date_two, max_cloud_covering)
            with concurrent.futures.ThreadPoolExecutor() as modis_caller:
                modis_caller.submit(modis.modissc_init)
                modis_caller.submit(modis.modistsr_init)
                modis_caller.submit(modis.modistta_init)           

        if datasets_other.value:
            others = o_init.OthersInitialization(m, date_one, date_two)   
            with concurrent.futures.ThreadPoolExecutor() as others_caller:
                others_caller.submit(others.firms_init)
                others_caller.submit(others.alostd_init)

    except ee.EEException as error:
        print(Fore.RED + '[GEE map plotting/initialization error] - {}'.format(str(error)))
        logger.error(error)
        pass

    if manual_input.value:
        try:
            custom_ds = mr.Text(value="-", label="[Custom DS] Enter GEE dataset name", rows=1)
            date_one_custom = mr.Text(value="1972-01-01", label="[Custom DS] Enter start date (format YYYY-MM-DD)", rows=1)
            date_two_custom = mr.Text(value="1973-01-01", label="[Custom DS] Enter final date (format YYYY-MM-DD)", rows=1)
            cd_init.customds_initialization(m, date_one_custom, date_two_custom, custom_ds)
        except ee.EEException as error:
            print(Fore.RED + '[GEE map plotting/initialization error] - {}'.format(str(error)))
            logger.error(error)
            pass

    m.add_text(text, **text_params)
        
    mr.Markdown(text = "#GOOGLE EARTH ENGINE MAP 🌐")
    mr.Markdown(text = "**DATASETS TIME PERIOD: from {} to {}**".format(date_one.value, date_two.value))
    
    if manual_input.value:
        mr.Markdown(text = "**CUSTOM DATASETS TIME PERIOD: from {} to {}**".format(date_one_custom.value, date_two_custom.value))
    
    if ds_info.value:
        mr.Markdown(text="**LANDSAT BANDS COMBINATIONS ⤵**")
        clb = Image.open('gee//data//clb.png')
        plt.figure(figsize=(7, 7))
        plt.imshow(clb)
        plt.axis('off')
        plt.show()
        mr.Markdown(text="**SENTINEL 2 BANDS COMBINATIONS ⤵**")
        csb = Image.open('gee//data//csb.png')
        plt.figure(figsize=(7, 7))
        plt.imshow(csb)
        plt.axis('off')
        plt.show()
        txtr.ds_info_read()
    
    display(m)

In [ ]:
# OSM PROCESSING

if control_panel_choice.value == "OpenStreetMap":
    mr.Markdown(text = "#OPENSTREETMAP 🌐")
    geojson_file = mr.File(label="Upload GeoJSON file for processing", max_file_size="100MB")
    basic_zoom = mr.Slider(value=15, label='Basic map zoom (less = more info on the map): ', min=1, max=20)
    add_datasets = mr.Checkbox(value=False, label="View additional maps")
    dual_maps = mr.Checkbox(value=False, label="Enter dual maps mode")
    mini_map = mr.Checkbox(value=False, label="Show mini map")

    try:
        map_folium = folium.Map(zoom_start=basic_zoom.value)
        if geojson_file.filepath is not None:
            try:
                geojson = open(geojson_file.filepath)
                geojson_info = json.load(geojson)
                coordinates = geojson_info['features'][0]['geometry']['coordinates']
                folium.GeoJson(geojson_info, name='geojson').add_to(map_folium)
            except IndexError as error:
                print(Fore.RED + '[OSM GeoJSON file error] - {}'.format(str(error)))
        else:
            pass
            
        if add_datasets.value:
            op.additional_datasets(map_folium)

        op.map_layers(map_folium)
        
        if dual_maps.value:
            map_folium = folium.plugins.DualMap(zoom_start=basic_zoom.value)
            op.dual_maps_processing(map_folium)
        else:
            folium.LayerControl().add_to(map_folium)

        draw = Draw(export=True)
        draw.add_to(map_folium)

        plugins.Geocoder(
            position='bottomleft',
            force_separate_button=True
        ).add_to(map_folium)

        plugins.Fullscreen(
            position='topleft',
            force_separate_button=False
        ).add_to(map_folium)

        if mini_map.value:
            plugins.MiniMap().add_to(map_folium)
        display(map_folium)
            
    except (ValueError, TypeError):
        print('OSM coordinates were not entered (or entered in incorrect type) or they are within geographical boundaries')